In [ ]:
import json
import pickle

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, mean_absolute_error, max_error

In [ ]:
file_name = "difficulty_predictions_gpt-3.5-turbo-0125_20240428-1323.pickle"
with open(file_name, 'rb') as f:
    list_of_json = pickle.load(f)
predictions = [json.loads(x) for x in list_of_json]

In [ ]:
len(predictions)

In [ ]:
target_data_path = "data/Annotations.csv"
df = pd.read_csv(target_data_path)

In [ ]:
df.head()

In [ ]:
preds = {
    "cfp": [],
    "cfd": [],
    "uc": [],
    "rhc": [],
    "cpt": [],
    "bd": [],
    "r": [],
}
ground_truth = {
    "cfp": [],
    "cfd": [],
    "uc": [],
    "rhc": [],
    "cpt": [],
    "bd": [],
    "r": [],
}
for i, pred in enumerate(predictions):
    for key in preds.keys():

        if key in pred.keys() and isinstance(pred[key], (int, float)):
            the_pred = pred[key]

            preds[key].append(the_pred)
            ground_truth[key].append(df.iloc[i][key.upper()])

In [ ]:
def boundary_loss(targets, preds):
        targets = np.asarray(targets)
        preds = np.asarray(preds)
        return np.mean(np.abs(targets - preds))

In [ ]:
rubric = {
    "CFP": "Chord finger positioning",
    "CFD": "Chord finger difficulty",
    "UC": "Uncommonness of chord",
    "RHC": "Right-hand complexity",
    "CPT": "Chord progression time",
    "BD": "Beat difficulty",
    "R": "Repetitiveness",
}

b_losses = {}
acc_losses = {}
print("Metric", "\t"*4, "Acc ", "MAE ", "MaxErr", "B-Loss", sep="\t")
for key in ground_truth.keys():
    # print(key)
    prrr = [round(pr) for pr in preds[key]]
    acc = accuracy_score(ground_truth[key], prrr)
    mae = mean_absolute_error(ground_truth[key], prrr)
    max_err = max_error(ground_truth[key], preds[key])
    b_loss = boundary_loss(ground_truth[key], preds[key])
    b_losses[key] = round(b_loss, 2)
    acc_losses[key] = round(acc, 2)
    print(f"{rubric[key.upper()]:25}\t{acc:.2f}\t{mae:.2f}\t{max_err:.2f}\t{b_loss:.2f}")
mean_acc = np.mean(list(acc_losses.values()))
mean_b_loss = np.mean(list(b_losses.values()))
print()
print(f"Average\tAcc: {mean_acc :.2f} B-Loss: {mean_b_loss :.2f}")
print(f"Aggregate\tB-Loss: {sum(b_losses.values()):.2f}")

In [ ]:
print(b_losses)

In [ ]:
print(list_of_json[0])

In [ ]:
filename_messages = "difficulty_predictions_gpt-3.5-turbo-1106_20231125-2318_messages.pickle"
with open(filename_messages, 'rb') as f:
    messages = pickle.load(f)

In [ ]:
print(messages[0])

In [ ]:
print(df.iloc[0])

In [ ]:
ground_truth["r"]